# Encode audio with a text encoder!

In [1]:
# Install necessary libraries
!pip install transformers datasets pandas sentencepiece

DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
import os
import pandas as pd
import pickle
from tqdm.auto import tqdm
from transformers import T5Tokenizer, T5Model, T5EncoderModel

/home/th716/.conda/envs/audiodiff_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:

csv_file_path = "../cache/spotify_sleep_dataset/csv/Sleep_FullDataset_withDuplicates.csv"
df = pd.read_csv(csv_file_path)

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5EncoderModel.from_pretrained(model_name)

def encode_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
import numpy as np

text1 = 'chillhop lo-fi beats'
text2 = 'sleep'
text3 = 'lullaby'
text4 = 'binaural'
text5 = 'compositional ambient'
text6 = 'ambient'
text7 = 'brain waves'
text8 = 'calming instrumental'


encoding1 = encode_text(text1)
encoding2 = encode_text(text2)
encoding3 = encode_text(text3)
encoding4 = encode_text(text4)
encoding5 = encode_text(text5)
encoding6 = encode_text(text6)
encoding7 = encode_text(text7)
encoding8 = encode_text(text8)

encodings = np.stack([encoding1, encoding2, encoding3, encoding4, encoding5, encoding6, encoding7, encoding8], axis=0).reshape(8, 1, 512)

with open('encodings_512.pkl', 'wb') as f:
    pickle.dump(encodings, f)

print("Encodings saved to 'encodings.pkl'")

Encodings saved to 'encodings.pkl'


In [6]:
encodings.shape

(8, 1, 768)

In [10]:

audio_dir = "../cache/spotify_sleep_dataset/waveform"

encodings = {}
for audio_file in tqdm(os.listdir(audio_dir)):
    if audio_file.endswith(".wav"):
        file_id = int(os.path.splitext(audio_file)[0])
        genre_text = df.loc[df.index == file_id, 'Genres'].values[0]
        encodings[audio_file] = encode_text(genre_text)

output_file = "../cache/spotify_sleep_dataset/genre_encodings.p"
with open(output_file, "wb") as f:
    pickle.dump(encodings, f)

print(f"Encodings saved to {output_file}")


  0%|          | 0/11485 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


100%|██████████| 11485/11485 [14:24<00:00, 13.28it/s]


Encodings saved to ../cache/spotify_sleep_dataset/genre_encodings.p


In [5]:
ssd_df = pd.read_csv(csv_file_path)